In [ ]:
from snowflake.snowpark.functions import sproc, udf, udtf, call_udf, col, lit,pandas_udf
from snowflake.snowpark.types import IntegerType, StructType, StructField, PandasSeriesType
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
import pandas as pd
import os

In [ ]:
os.environ['SNOWFLAKE_ACCOUNT'] = 'EBXSMJF-PL33393'
os.environ['SNOWFLAKE_USER'] = 'ritishadhikari'
os.environ['SNOWFLAKE_PASSWORD'] = 'Eklavya@05072024'
os.environ['SNOWFLAKE_DATABASE'] = 'test'
os.environ['SNOWFLAKE_SCHEMA'] = 'public'

In [ ]:
session=Session.builder.configs(SnowflakeLoginOptions()).create()
session.query_tag='func-gen'

Anonymous Stored Procedure (with Lambda)

In [ ]:
# create a temporary procedure
add_one=sproc(
    func=lambda session, x: session.sql(f"SELECT {x}+1").collect()[0][0],
    input_types=[IntegerType()],
    return_type=IntegerType(),
    # packages=["snowflake-snowpark-python==1.13.0"],
    session=get_active_session()
)
ret=add_one(1)
print(f"add_one:{ret}")

In [ ]:
add_two=sproc(
    func=lambda session, x: session.sql(f"SELECT {x}+2").collect()[0][0],
    name='add_two_proc',
    replace=True,
    input_types=[IntegerType()],
    return_type=IntegerType(),
    packages=['snowflake-snowpark-python'],
    session=get_active_session()
)
ret=session.call('add_two_proc',23)
print(f"add_two:{ret}")

In [ ]:
CREATE OR REPLACE FILE FORMAT TEST.PUBLIC.CSV_HEADER_PARSE
    TYPE='CSV' PARSE_HEADER=TRUE FIELD_OPTIONALLY_ENCLOSED_BY = '"';



-- PUT file:data/titanic.csv @TEST.PUBLIC.INT_STAGE
--     OVERWRITE=true AUTO_COMPRESS=false;

In [ ]:
CREATE OR REPLACE STAGE TEST.PUBLIC.INT_STAGE
    FILE_FORMAT=TEST.PUBLIC.CSV_HEADER_PARSE;

In [ ]:
session.file.put(
    'titanic.csv',
    '@TEST.PUBLIC.INT_STAGE',
    auto_compress=False,
    overwrite=True
)

In [ ]:
# this will create a permanent stored procedure
@sproc(
    name='add_three',
    replace=True,
    is_permanent=True,
    stage_location="@int_stage",
    packages=['snowflake-snowpark-python'],
    session=get_active_session()
)
def  add_three(session: Session, x:int) -> int:
    return session.sql(f"SELECT {x}+3").collect()[0][0]

ret=session.sql('CALL add_three(1)').collect()[0][0]
print(f"add_three: {ret}")

In [ ]:
# this is a temporary function
add_five=udf(func=lambda x: x+5,
             input_types=[IntegerType()],
             return_type=IntegerType())

df=session.create_dataframe([[1,],]).to_df("a")
ret=df.select(add_five(col('a'))).collect()[0][0]
print(f"add_five:{ret}")

In [ ]:
# this is a permanent function
add_six=udf(func=lambda x: x+5,
             name="add_six",
             replace=True,
             session=get_active_session(),
             input_types=[IntegerType()],
             return_type=IntegerType())

df=session.create_dataframe([[7,],]).to_df("a")
ret=df.select(add_six(col('a'))).collect()[0][0]
print(f"add_six:{ret}")

Registered UDF

In [ ]:
# register an udf
@udf(
name="add_nine",
replace=True,
is_permanent=True,
stage_location="@int_stage"
)
def add_nine(x:int)->int:
    return x+9

df=session.create_dataframe([[1]], schema=["a"])
ret=df.select(call_udf("add_nine",col("a"))).collect()[0][0]
print(f"add_nine:{ret}")

In [ ]:
# Registered UDTF

class GETTwo:
    def process(self, n):
        yield(1,)
        yield(n,)

get_two=udtf(handler=GETTwo, 
             output_schema=StructType([
                 StructField(
                    column_identifier="number",
                    datatype=IntegerType()
                    )
                ]),
             input_types=[IntegerType()]
            )
ret=session.table_function(get_two(lit(literal=3))).collect()
print(f"get_two: {ret}")

Pandas UDF

In [ ]:
@pandas_udf(
    name='add_eight',
    replace=True,
    input_types=[
            PandasSeriesType(element_type=IntegerType()),
            PandasSeriesType(element_type=IntegerType())
    ],
    return_type=PandasSeriesType(element_type=IntegerType()),
    session=get_active_session()
)
def add_eight(col1:pd.Series, col2: pd.Series) ->pd.Series:
    return col1+col2+8

df=session.create_dataframe([
    [1,2],
    [3,4],
    ],
    schema=["a","b"])
ret=df.select(call_udf("add_eight", col('a'), col('b'))).collect()

print(f"add_eight: {ret[0][0]}, {ret[1][0]}")